<a href="https://colab.research.google.com/github/Yarrieta/Entregas_PNL/blob/Desafio2/2c_Custom_embedding_con_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [29]:
!pip install PyPDF2

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PyPDF2 import PdfReader
from io import StringIO, BytesIO

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [30]:
# URL del archivo PDF
url = 'https://www.textos.info/jane-austen/orgullo-y-prejuicio/pdf'

# Descargar el archivo PDF
response = requests.get(url)
response.raise_for_status()  # Verificar que la solicitud fue exitosa

# Leer el archivo PDF
pdf_file = BytesIO(response.content)
pdf_reader = PdfReader(pdf_file) # Use PdfReader here

# Imprimir el número de páginas del PDF
num_pages = len(pdf_reader.pages) # Use pages attribute to get number of pages
#print(f'El PDF tiene {num_pages} páginas.')

listOfList = []
# Leer y mostrar el contenido de cada página
for page_num in range(num_pages):
    page = pdf_reader.pages[page_num] # Access pages using index
    text = page.extract_text()
    #rowList = [page_num + 1,text]
    rowList = [text]
    listOfList.append(rowList)
    #print(f'Contenido de la página {page_num + 1}:\n{text}\n')
df = pd.DataFrame(listOfList, columns=['Contenido'])

In [31]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df.head()

,Contenido
0,Orgullo y Prejuicio\nJane Austen\ntextos.infoB...
1,Texto núm. 3006\nTítulo: Orgullo y Prejuicio\n...
2,Capítulo I\nEs una verdad mundialmente reconoc...
3,"—¿Está casado o soltero?\n—¡Oh!, soltero, quer..."
4,alegrará mucho de veros; y tú le llevarás unas...


In [32]:
df.replace(r'\n', ' ', regex=True, inplace=True)

In [33]:
df

,Contenido
0,Orgullo y Prejuicio Jane Austen textos.infoBib...
1,Texto núm. 3006 Título: Orgullo y Prejuicio Au...
2,Capítulo I Es una verdad mundialmente reconoci...
3,"—¿Está casado o soltero? —¡Oh!, soltero, queri..."
4,alegrará mucho de veros; y tú le llevarás unas...
...,...
348,boca sin decir una vulgaridad. Ni siquiera aqu...
349,Capítulo LXI El día en que la señora Bennet se...
350,moralizar acerca de todas las visitas de las m...
351,progresar en su carrera por consideración a El...


In [34]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 353


### 1 - Preprocesamiento

In [35]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [54]:
# Demos un vistazo
sentence_tokens[2:4]

[['capítulo',
  'i',
  'es',
  'una',
  'verdad',
  'mundialmente',
  'reconocida',
  'que',
  'un',
  'hombre',
  'soltero',
  'poseedor',
  'de',
  'una',
  'gran',
  'fortuna',
  'necesita',
  'una',
  'esposa',
  'sin',
  'embargo',
  'poco',
  'se',
  'sabe',
  'de',
  'los',
  'sentimientos',
  'u',
  'opiniones',
  'de',
  'un',
  'hombre',
  'de',
  'tales',
  'condiciones',
  'cuando',
  'entra',
  'a',
  'formar',
  'parte',
  'de',
  'un',
  'vecindario',
  'esta',
  'verdad',
  'está',
  'tan',
  'arraigada',
  'en',
  'las',
  'mentes',
  'de',
  'algunas',
  'de',
  'las',
  'familias',
  'que',
  'lo',
  'rodean',
  'que',
  'algunas',
  'le',
  'consideran',
  'de',
  'su',
  'legítima',
  'propiedad',
  'y',
  'otras',
  'de',
  'la',
  'de',
  'sus',
  'hijas',
  '—mi',
  'querido',
  'señor',
  'bennet',
  '—le',
  'dijo',
  'un',
  'día',
  'su',
  'esposa—',
  '¿sabías',
  'que',
  'por',
  'fin',
  'se',
  'ha',
  'alquilado',
  'netherfield',
  'park',
  'el',
  

### 2 - Crear los vectores (word2vec)

In [37]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [38]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [39]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [40]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 353


In [41]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2223


### 3 - Entrenar embeddings

In [42]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1003320.3125
Loss after epoch 1: 714922.4375
Loss after epoch 2: 659916.75
Loss after epoch 3: 603292.0
Loss after epoch 4: 599299.75
Loss after epoch 5: 596668.75
Loss after epoch 6: 563251.5
Loss after epoch 7: 560178.5
Loss after epoch 8: 554465.5
Loss after epoch 9: 552330.0
Loss after epoch 10: 550415.0
Loss after epoch 11: 546641.0
Loss after epoch 12: 544435.5
Loss after epoch 13: 532355.0
Loss after epoch 14: 522116.0
Loss after epoch 15: 519385.0
Loss after epoch 16: 517394.0
Loss after epoch 17: 514326.0
Loss after epoch 18: 515625.0
Loss after epoch 19: 515790.0


(1336403, 2311720)

### 4 - Ensayar

In [44]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["orgullo"], topn=10)

[('atento', 0.7783492207527161),
 ('supuesto', 0.7686362862586975),
 ('ardiente', 0.7682057023048401),
 ('futuro', 0.7599228620529175),
 ('culpable', 0.7540954947471619),
 ('abominable', 0.7529197931289673),
 ('sincero', 0.7508373856544495),
 ('virtud', 0.7462146282196045),
 ('entendimiento', 0.7452846765518188),
 ('generoso', 0.74217689037323)]

In [46]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["orgullo"], topn=10)

[('casa', -0.106223925948143),
 ('iba', -0.13004815578460693),
 ('al', -0.13123661279678345),
 ('pudo', -0.15213067829608917),
 ('después', -0.1581754833459854),
 ('quiso', -0.16508065164089203),
 ('señorita', -0.16989107429981232),
 ('unos', -0.17397640645503998),
 ('minutos', -0.17573705315589905),
 ('a', -0.18497316539287567)]

In [48]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["prejuicio"], topn=10)

[('ardiente', 0.8756892085075378),
 ('tuviera', 0.8659945726394653),
 ('supuesto', 0.8635374903678894),
 ('profunda', 0.8559527397155762),
 ('siente', 0.8546525239944458),
 ('dejaba', 0.8513489961624146),
 ('afortunada', 0.8498439788818359),
 ('escena', 0.8493443131446838),
 ('personalmente', 0.8484236001968384),
 ('hallarse', 0.843302309513092)]

In [55]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mujeres"], topn=5)

[('elegantes', 0.8999901413917542),
 ('familias', 0.8422377109527588),
 ('objeciones', 0.8303874135017395),
 ('encantadoras', 0.8269631862640381),
 ('mejores', 0.8260461688041687)]

In [56]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["mujeres"])

[('al', -0.13829684257507324),
 ('puerta', -0.14134450256824493),
 ('hacia', -0.15525639057159424),
 ('iba', -0.15613055229187012),
 ('entró', -0.15711623430252075),
 ('sin', -0.15946805477142334),
 ('hermano', -0.16906100511550903),
 ('hasta', -0.17698118090629578),
 ('día', -0.17733952403068542),
 ('abrió', -0.18324212729930878)]

In [57]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("darcy")
print(vector_love)

[-1.76617146e-01  2.12886259e-01 -1.50689751e-01 -1.60423830e-01
 -1.29020303e-01  8.41309130e-02  2.08450198e-01  3.11989486e-01
  2.92697191e-01 -1.83828726e-01  2.71546721e-01  1.05436537e-02
  3.58829141e-01 -1.26206279e-01 -2.00890258e-01  2.50738502e-01
  1.66455768e-02  1.85242012e-01 -1.50624737e-01 -1.30918041e-01
  8.44507590e-02 -1.38586447e-01 -2.03314751e-01 -2.33152173e-02
  1.50503695e-01  7.04772472e-02 -1.70743875e-02 -3.54567207e-02
 -1.25102028e-01 -3.11645657e-01  6.68587908e-02  3.02712739e-01
 -2.65852898e-01  3.92730415e-01  3.40334982e-01  1.16878167e-01
 -3.01449392e-02 -1.61326975e-01 -1.06162958e-01 -1.14204220e-01
 -4.02894020e-02 -1.36806101e-01 -1.73419923e-01 -1.00543192e-02
 -7.87438303e-02 -1.28052067e-02  3.51404339e-01  1.62036158e-02
 -1.12720415e-01 -8.88348818e-02  4.78549004e-02  7.44358748e-02
  2.43005142e-01  1.65399581e-01  1.34832859e-01  8.87597203e-02
  1.60046533e-01  1.90964252e-01 -9.11093876e-02  2.04837099e-01
  7.58494437e-02 -1.48268

In [58]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('darcy', 1.0),
 ('horror', 0.6584476232528687),
 ('bingley', 0.622105062007904),
 ('atentamente', 0.6163091659545898),
 ('abajo', 0.6043081879615784),
 ('observaba', 0.5998933911323547),
 ('horrible', 0.5988080501556396),
 ('tratar', 0.589838445186615),
 ('reconoció', 0.5841003656387329),
 ('aquello', 0.5750672817230225)]

In [60]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darcy"], topn=10)

[('horror', 0.6584476232528687),
 ('bingley', 0.622105062007904),
 ('atentamente', 0.6163091659545898),
 ('abajo', 0.6043082475662231),
 ('observaba', 0.5998933911323547),
 ('horrible', 0.5988080501556396),
 ('tratar', 0.589838445186615),
 ('reconoció', 0.5841003656387329),
 ('aquello', 0.5750672817230225),
 ('fijó', 0.5713756084442139)]

### 5 - Visualizar agrupación de vectores

In [61]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [62]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [63]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [64]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

#Bibliografía

promt1: como leer un archivo pdf desde una url
fecha: 19/07/2024